# MODELO DE RECOMENDACION DE PELICULAS

Primero vamos a trabajar el modelo de Machine Learning en un Notebook, para posteriormente incluirlo en el script main.py 

In [ ]:
# Librerias
import numpy as np
import pandas as pd
import seaborn as sns
sns.set
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

In [ ]:
# Cargar la Informacion
# Función que carga los datos desde un archivo Parquet
def cargar_datos():
    df_data_filtrada = pd.read_parquet('../data_filtrada.parquet')
    return df_data_filtrada

#Cargo los Datos
data_filtrada = cargar_datos()

In [ ]:
#  vectorizo la información de texto
tfidf = TfidfVectorizer(stop_words='english')    #Instancio
tfidf_matrix = tfidf.fit_transform(data_filtrada['texto_combinado'])  #vectorizo


In [ ]:
# Estandarizo los datos numericos a usar:
scaler = StandardScaler()  #Instancio 
data_filtrada[['popularity','release_year']] = scaler.fit_transform(data_filtrada[['popularity', 'release_year']]) #escalo

In [ ]:
#Creo una matrix total uniendo las catacteristicas categóricas más las numéricas.
matrix_total = np.hstack((tfidf_matrix.toarray(), data_filtrada[['popularity', 'release_year']].values))

In [ ]:
# Convierto tfidf_matrix a array y obtengo los valores de las columnas numéricas
tfidf_array = tfidf_matrix.toarray()
numeric_data = data_filtrada[['popularity', 'release_year']].values

In [ ]:
# Configura el número de componentes principales (puedes ajustar esto según sea necesario)
n_components = 50  # Ajusta según tu caso específico

# Aplica PCA para reducir la dimensionalidad
pca = PCA(n_components=n_components)
matrix_reduced = pca.fit_transform(matrix_total)

# Calcula la matriz de similitud con la matriz reducida
similarity_matrix = cosine_similarity(matrix_reduced, matrix_reduced)

In [ ]:
def recomendacion(titulo: str):
    try:
        idx_list = data_filtrada.index[data_filtrada['title'] == titulo].tolist()
        
        if not idx_list:
            return {'error': "Película no encontrada"}
        
        idx = idx_list[0]
        
        print('Titulo de la Película:', titulo)  # Log de depuración
        print('Índice:', idx)  # Log de depuración
        print('Tipo de índice:', type(idx))

        # Obtengo los pares de películas índice y score
        similarity_score = list(enumerate(similarity_matrix[idx]))

        # Ordeno las películas por puntaje de similitud
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)

        # Tomo los índices de las 5 películas similares. Quito la primera [0] que es la misma dada
        similar_movies_indices = [i[0] for i in similarity_score[1:6]]

        recomendadas = data_filtrada['title'].iloc[similar_movies_indices].tolist()
    
    except Exception as e:
        return {'error': f"Error interno: {str(e)}"}
    
    return {'Recomendaciones': recomendadas}


In [ ]:
data_filtrada['title'].head()


In [ ]:
pelicula = 'two friends'
recomendacion(pelicula)